In [15]:
import numpy as np
from scipy.integrate import odeint
import plotly.graph_objs as go
from plotly.subplots import make_subplots  # Import make_subplots for creating subplots

# Define the differential equations
def dS_dt(S, I, lambda_S, lambda_R, fq):
    return -lambda_S * S * fq + lambda_R * I

def dE_dt(S, E, lambda_S, fq):
    return lambda_S * S * fq - lambda_E * E

def dI_dt(I, E, lambda_E, lambda_R):
    return lambda_E * E - lambda_R * I

# Define the function for the probability f(q(t)) (Poisson distribution)
def f_q(x, lower_rate, higher_rate):
    # 5/6 of people have lower rate, 1/6 have higher rate
    # Calculate probability of having higher rate
    p_higher = 1 / 6
    p_lower = 1 - p_higher
    
    # Use Poisson distribution to determine probability f(q(t))
    rate = p_lower * lower_rate + p_higher * higher_rate
    return 1 - np.exp(-rate * x)  # Example Poisson distribution

# Function to integrate the differential equations
def simulate_virus_spread(S0, E0, I0, lambda_S, lambda_E, lambda_R, lower_rate, higher_rate, total_time, dt):
    num_steps = int(total_time / dt) + 1
    t = np.linspace(0, total_time, num_steps)

    # Initial conditions
    y0 = [S0, E0, I0]

    # Parameters
    args = (lambda_S, lambda_E, lambda_R, lower_rate, higher_rate)

    # Define the ODE system
    def system(y, t, *args):
        S, E, I = y
        lambda_S, lambda_E, lambda_R, lower_rate, higher_rate = args
        fq = f_q(I, lower_rate, higher_rate)
        dS = dS_dt(S, I, lambda_S, lambda_R, fq)
        dE = dE_dt(S, E, lambda_S, fq)
        dI = dI_dt(I, E, lambda_E, lambda_R)  # Pass E, lambda_E, lambda_R to dI_dt
        return [dS, dE, dI]

    # Integrate the ODE system
    solution = odeint(system, y0, t, args=args)

    return t, solution[:, 0], solution[:, 1], solution[:, 2]

# Parameters for simulation
S0 = 2000000  # Initial number of susceptible peers
E0 = 10000    # Initial number of exposed peers
I0 = 1     # Initial number of infected peers
lambda_S = 0.035  # Rate of new file downloads per minute
lambda_E = 0.035  # Rate of becoming infected per minute
lambda_R = 0.007 # Rate of recovery per minute
lower_rate = 0.00005  # Lower infection rate for 5/6 of people
higher_rate = 0.0005  # Higher infection rate for 1/6 of people
total_time = 600    # Total simulation time
dt = 0.1            # Time step for integration

# Run simulation
t, S, E, I = simulate_virus_spread(S0, E0, I0, lambda_S, lambda_E, lambda_R, lower_rate, higher_rate, total_time, dt)

# Create Plotly figure with subplots
fig = make_subplots(rows=1, cols=1)

# Add traces for S, E, I
fig.add_trace(go.Scatter(x=t, y=S, mode='lines', name='Susceptible'), row=1, col=1)
fig.add_trace(go.Scatter(x=t, y=E, mode='lines', name='Exposed'), row=1, col=1)
fig.add_trace(go.Scatter(x=t, y=I, mode='lines', name='Infected'), row=1, col=1)

# Update layout
fig.update_layout(title='Virus Spread Simulation with Varied Poisson',
                  xaxis_title='Time',
                  yaxis_title='Population',
                  legend=dict(x=0, y=1, traceorder='normal'),
                  template = 'plotly_white')

# Show plot
fig.show()


